In [225]:
import pandas as pd
import numpy as np

In [ ]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from config import (
    N_CUSTOMERS,
    NUM_WEEKS,
    K,
    DECAY_RATE,
    RECOVERY_RATE,
    COOLDOWN_WINDOW,
    COOLDOWN_PENALTY,
    REGRET_THRESHOLD,
    SEEDS
)

In [ ]:
def initialize_world():
    df = pd.DataFrame({
        "Customer_id" : range(N_CUSTOMERS),
        "tolerance" : np.random.uniform(0,1.5,size=N_CUSTOMERS)
    })
    df['time_gap'] = np.random.exponential(scale=df['tolerance']*5)
    df['worsening_streak'] = 0
    df['urgency_score'] = 0
    df['reviewed'] = False
    reviewed_ids = np.random.choice(df.index, size = K, replace = False)
    df.loc[reviewed_ids,'reviewed'] = True

    df.loc[df['reviewed'],'tolerance'] = (df.loc[df['reviewed'],'tolerance']*(1+RECOVERY_RATE))

    df.loc[~df['reviewed'],'tolerance'] = (df.loc[~df['reviewed'],'tolerance']*(1-DECAY_RATE))

    df['prev_time_gap'] = df['time_gap']
    df['time_gap'] = np.random.exponential(scale=df['tolerance'] * 5)
    df['gap_change'] = df['prev_time_gap'] - df['time_gap']
    df.loc[df['gap_change'] > 0,'worsening_streak'] += 1
    df.loc[df['gap_change'] <= 0,'worsening_streak'] = 0
    df['urgency_score'] = df['gap_change'] * df['worsening_streak']
    return df

# NOTE:
### urgency_score computed in week t is used to decide reviews in week t+1
### this avoids circular dependency and mirrors real-world decision latency

In [228]:
def advance_one_week(df,decay_rate,recovery_rate):
    df.loc[df["reviewed"], "tolerance"] *= (1 + recovery_rate)
    df.loc[~df["reviewed"], "tolerance"] *= (1 - decay_rate)

    df["prev_time_gap"] = df["time_gap"]
    df["time_gap"] = np.random.exponential(scale=df["tolerance"] * 5)

    df["gap_change"] = df["prev_time_gap"] - df["time_gap"]

    df.loc[df["gap_change"] > 0, "worsening_streak"] += 1
    df.loc[df["gap_change"] <= 0, "worsening_streak"] = 0

    df["urgency_score"] = df["gap_change"] * df["worsening_streak"]

    return df

In [ ]:
df = initialize_world()
for week in range(NUM_WEEKS):
    df['reviewed'] = False
    to_review = (
        df.sort_values('urgency_score',ascending=False).head(K).index
    )
    df.loc[to_review,'reviewed'] = True
    df = advance_one_week(df,DECAY_RATE,RECOVERY_RATE)

    print(
        f"Week {week + 2} |"
        f"Max Streak : {df['worsening_streak'].max():.0f} |"
        f"Max Urgency : {df['urgency_score'].max():.2f}"
    )

Week 2 |Max Streak : 2 |Max Urgency : 12.41
Week 3 |Max Streak : 3 |Max Urgency : 17.10
Week 4 |Max Streak : 4 |Max Urgency : 20.37
Week 5 |Max Streak : 4 |Max Urgency : 21.26
Week 6 |Max Streak : 3 |Max Urgency : 22.62
Week 7 |Max Streak : 4 |Max Urgency : 23.06
Week 8 |Max Streak : 3 |Max Urgency : 22.10
Week 9 |Max Streak : 3 |Max Urgency : 14.06
Week 10 |Max Streak : 4 |Max Urgency : 36.35
Week 11 |Max Streak : 3 |Max Urgency : 8.36


In [230]:
print(df.sort_values('urgency_score',ascending=False)[['worsening_streak','gap_change','urgency_score']].head(5)
      )

    worsening_streak  gap_change  urgency_score
36                 3    2.785995       8.357985
64                 1    8.204796       8.204796
91                 1    7.970292       7.970292
6                  2    3.626277       7.252553
0                  2    3.538325       7.076651
